<a href="https://colab.research.google.com/github/raajmalhotra143/Android-apps/blob/main/Learning/DSA_by_AC_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Setup for running c++ codes**


In [3]:
from IPython.core.magic import register_cell_magic
import subprocess
import os
import re # Import re for regex
import IPython # Import IPython module

@register_cell_magic
def cpp(line, cell):
    # Regex to find file blocks and extract filenames
    file_blocks_raw = re.split(r'^\s*--- FILE:\s*(\S+\.cpp)\s*---\s*$', cell, flags=re.MULTILINE)

    filenames_and_contents = {}
    main_filename_for_input_check = ""

    if len(file_blocks_raw) == 1:
        # No '--- FILE:' header found, treat the whole cell as a single C++ file.
        filenames_and_contents["main.cpp"] = cell.strip()
        main_filename_for_input_check = "main.cpp"
    elif not file_blocks_raw[0].strip(): # Starts with --- FILE:
        # The first element is empty, then filename, then content, then filename, etc.
        # [ '', 'file1.cpp', 'content1', 'file2.cpp', 'content2', ... ]
        for i in range(1, len(file_blocks_raw), 2):
            filename = file_blocks_raw[i].strip()
            content = file_blocks_raw[i+1].strip()
            filenames_and_contents[filename] = content
        main_filename_for_input_check = list(filenames_and_contents.keys())[-1] # Last file is main
    else:
        print("❌ Invalid C++ cell format. Ensure each file starts with '--- FILE: <filename.cpp> ---' or provide a single file without headers.")
        return

    if not filenames_and_contents:
        print("❌ No C++ code found in the cell or incorrect file block format.")
        return

    executable_name = "solution" # Fixed executable name for the final linked program

    object_files = []
    source_files = []

    # 1. Save C++ code to files and compile into object files
    print("--- COMPILING C++ FILES ---")
    for filename, cpp_code in filenames_and_contents.items():
        if filename.endswith(".cpp"):
            obj_filename = filename.replace(".cpp", ".o")
            source_files.append(filename)
            object_files.append(obj_filename)

            with open(filename, "w") as f:
                f.write(cpp_code)

            compile_process = subprocess.run(["g++", "-c", filename, "-o", obj_filename],
                                             capture_output=True, text=True)

            if compile_process.returncode != 0:
                print(f"❌ Compilation Failed for {filename}!")
                print(compile_process.stderr)
                # Clean up generated files before exiting
                for f in source_files:
                    if os.path.exists(f): os.remove(f)
                for f in object_files:
                    if os.path.exists(f): os.remove(f)
                return
            else:
                print(f"✅ Compilation Successful for {filename}!")
        else:
            print(f"Skipping non-C++ file: {filename}")

    # 2. Link all object files to create the executable
    print("\n--- LINKING OBJECT FILES ---")
    link_command = ["g++"] + object_files + ["-o", executable_name]
    link_process = subprocess.run(link_command, capture_output=True, text=True)

    if link_process.returncode != 0:
        print("❌ Linking Failed!")
        print(link_process.stderr)
        # Clean up generated files
        for f in source_files:
            if os.path.exists(f): os.remove(f)
        for f in object_files:
            if os.path.exists(f): os.remove(f)
        return
    else:
        print(f"✅ Linking Successful! Executable: {executable_name}")

    # 3. Handle input for the executable
    user_input = ""
    input_delimiter = "---INPUT---"
    interactive_input_needed = False

    # Check for specific input section in the *original full cell content*
    if input_delimiter in cell:
        parts = cell.split(input_delimiter, 1)
        # Input part is after the delimiter
        user_input = parts[1].strip()
    else:
        # Check if the designated 'main' file uses cin for interactive input
        if re.search(r'\bcin\s*>>', filenames_and_contents.get(main_filename_for_input_check, "")):
            print(f"Detected 'cin' in {main_filename_for_input_check}.")
            print("Please provide *all* required inputs for the C++ program below, one value per line.")
            print("To signal that you have provided all inputs, enter an empty line (just press Enter) after a prompt.")
            print("Alternatively, you can press Ctrl+D for EOF (End Of File) to finish input.")

            collected_input_lines = []
            while True:
                try:
                    # Modified prompt for clarity
                    line_input = input("C++ Input (type empty line to finish)> ")
                    if not line_input:
                        break
                    collected_input_lines.append(line_input)
                except EOFError:
                    print("\nEOF received. Finishing input.")
                    break
            user_input = "\n".join(collected_input_lines)
            if user_input:
                interactive_input_needed = True

    # 4. Run the executable
    print("\n--- RUNNING PROGRAM ---")
    # If user_input was collected interactively or via ---INPUT---
    # We rely on the C++ program itself to echo the inputs if designed to.
    if user_input:
        # Do not print a summary of inputs here anymore,
        # as the C++ program is now expected to print its own prompts and echo values.
        pass
    else: # if no user_input was defined via ---INPUT--- or collected interactively
        # But still check if C++ code might implicitly ask for input (cin without ---INPUT---)
        if not re.search(r'\bcin\s*>>', filenames_and_contents.get(main_filename_for_input_check, "")):
            print("No input provided to C++ program.\n")

    run_process = subprocess.run([f"./{executable_name}"],
                                 input=user_input,
                                 capture_output=True,
                                 text=True)

    # 5. Display Output
    # The output from the C++ program already contains the prompts and echoed inputs.
    print(run_process.stdout)
    if run_process.stderr:
        print("Runtime Errors:", run_process.stderr)

    # 6. Clean up generated files
    print("\n--- CLEANING UP ---")
    if os.path.exists(executable_name):
        os.remove(executable_name)
        print(f"Removed executable: {executable_name}")
    for f in source_files:
        if os.path.exists(f):
            os.remove(f)
            print(f"Removed source file: {f}")
    for f in object_files:
        if os.path.exists(f):
            os.remove(f)
            print(f"Removed object file: {f}")

# Re-register the magic, in case it was already registered.
ip = IPython.get_ipython()
if ip and 'cpp' in ip.magics_manager.magics['cell']:
    del ip.magics_manager.magics['cell']['cpp']
if ip: # Only register if ip is not None
    register_cell_magic(cpp)


## Task
The agent has successfully generated and provided the `Lecture3_Full_Report.ipynb` notebook as requested.

The task is completed.


## Acknowledge Limitation

### Subtask:
Inform the user that directly reading and extracting content from specific pages of a PDF is not a supported operation.


Unfortunately, directly reading and extracting content from specific pages of a PDF document is not a supported operation in this environment. The current tools do not have built-in PDF parsing capabilities that allow for granular content extraction from particular page ranges.

## Final Task

### Subtask:
Conclude the interaction by explaining the limitation and asking if there's anything else that can be assisted with.


## Summary:

### Data Analysis Key Findings
* The environment's current tools do not support directly reading and extracting content from specific pages of a PDF document.
* There is a lack of built-in PDF parsing capabilities that would allow for granular content extraction from particular page ranges within the existing toolset.

### Insights or Next Steps
* Acknowledge the identified limitation regarding PDF content extraction to the user.


# **Lecture 2: Variables, Data Types & Operators | DSA Series (C++)**

This notebook covers every detail from the lecture, including memory concepts, variable rules, and operator logic.

---


setrup





## **1. Boilerplate Code (The Skeleton of a C++ Program)**

Every C++ program must follow a specific structure to work. This is called "Boilerplate Code."

* `#include <iostream>`: This is a **Preprocessor Directive**. It tells the compiler to include the `iostream` (Input-Output Stream) file. Without this, the computer does not know what `cout` (print) or `cin` (input) means.
* `using namespace std;`: The C++ standard library uses a namespace called `std`. Without this line, you would have to write `std::cout` every time. This line allows us to just write `cout`.
* `int main() { ... }`: This is the **Main Function**. The execution of every C++ program begins here. It returns an integer (`int`).
* `return 0;`: This signals that the program has finished running successfully. `0` usually means "no error".

In [2]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    // 'cout' stands for Console Output
    // 'endl' stands for End Line (moves cursor to next line)
    cout << "Apna College" << endl;

    // '\n' is another way to create a new line
    cout << "Hello World!" << "\n";

    return 0;
}

--- COMPILING C++ FILES ---
✅ Compilation Successful for main.cpp!

--- LINKING OBJECT FILES ---
✅ Linking Successful! Executable: solution

--- RUNNING PROGRAM ---
No input provided to C++ program.

Apna College
Hello World!


--- CLEANING UP ---
Removed executable: solution
Removed source file: main.cpp
Removed object file: main.o


In [3]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    // 'cout' stands for Console Output
    // 'endl' stands for End Line (moves cursor to next line)
    cout << "Apna College" << endl;

    // '\n' is another way to create a new line
    cout << "Hello World!" << "\n";

    return 0;
}

--- COMPILING C++ FILES ---
✅ Compilation Successful for main.cpp!

--- LINKING OBJECT FILES ---
✅ Linking Successful! Executable: solution

--- RUNNING PROGRAM ---
No input provided to C++ program.

Apna College
Hello World!


--- CLEANING UP ---
Removed executable: solution
Removed source file: main.cpp
Removed object file: main.o


---
## **2. Variables and Data Types**

A **Variable** is essentially a name given to a memory location. Think of it as a container or a box where we store data.

### **Rules for Naming Variables (Identifiers)**
1.  Must start with a letter (a-z, A-Z) or an underscore (`_`).
2.  Cannot start with a number (e.g., `1name` is invalid).
3.  No spaces allowed (e.g., `my age` is invalid; use `my_age` or `myAge`).
4.  C++ is **Case Sensitive** (`age`, `Age`, and `AGE` are three different variables).

### **Primitive Data Types**
| Data Type | Description | Size in Memory | Example |
| :--- | :--- | :--- | :--- |
| `int` | Integers (Whole numbers) | 4 Bytes (32 bits) | `int age = 25;` |
| `char` | Character (Single letter) | 1 Byte (8 bits) | `char grade = 'A';` |
| `float` | Floating Point (Small decimal) | 4 Bytes | `float pi = 3.14f;` |
| `bool` | Boolean (True or False) | 1 Byte | `bool isSafe = true;` |
| `double` | Double Precision (Large decimal) | 8 Bytes | `double price = 100.99;` |


In [4]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    int age = 25;
    char grade = 'A';
    float pi = 3.14f;
    bool isSafe = true;
    double price = 100.99;

    cout << "Age value: " << age << endl;
    cout << "Grade value: " << grade << endl;
    cout << "PI value: " << pi << endl;
    cout << "Is Safe (1=true, 0=false): " << isSafe << endl;
    cout << "Price value: " << price << endl;

    #// Checking the size of an int in memory
    cout << "Size of int: " << sizeof(age) << " bytes" << endl;

    return 0;
}

--- COMPILING C++ FILES ---
✅ Compilation Successful for main.cpp!

--- LINKING OBJECT FILES ---
✅ Linking Successful! Executable: solution

--- RUNNING PROGRAM ---
No input provided to C++ program.

Age value: 25
Grade value: A
PI value: 3.14
Is Safe (1=true, 0=false): 1
Price value: 100.99
Size of int: 4 bytes


--- CLEANING UP ---
Removed executable: solution
Removed source file: main.cpp
Removed object file: main.o


---
## **3. Type Casting**

Type casting is the process of converting a variable from one data type to another.

1.  **Implicit Conversion:** Done automatically (Small -> Large type).
2.  **Explicit Conversion:** Forced by programmer (Large -> Small type).


In [5]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    // 1. Implicit Conversion (Char -> Int)
    char grade = 'a'; // ASCII value of 'a' is 97
    int value = grade;
    cout << "ASCII value of 'a': " << value << endl;

    // 2. Explicit Conversion (Double -> Int)
    double bigNumber = 100.99;
    int smallNumber = (int)bigNumber; // Manually casting to int
    cout << "Original Double: " << bigNumber << endl;
    cout << "Casted Int (Decimals lost): " << smallNumber << endl;

    return 0;
}

--- COMPILING C++ FILES ---
✅ Compilation Successful for main.cpp!

--- LINKING OBJECT FILES ---
✅ Linking Successful! Executable: solution

--- RUNNING PROGRAM ---
No input provided to C++ program.

ASCII value of 'a': 97
Original Double: 100.99
Casted Int (Decimals lost): 100


--- CLEANING UP ---
Removed executable: solution
Removed source file: main.cpp
Removed object file: main.o


---
## **4. Input in C++ (Special Colab Instructions)**

**Issue:** You cannot type interactively in Colab while C++ code runs.
**Solution:** We create a text file (`input.txt`) and feed it to the program.

In [4]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    int age;
    cout << "Enter your age: ";
    cin >> age; // Read age from input

    cout << "Your age is: " << age << endl;
    return 0;
}

--- COMPILING C++ FILES ---
✅ Compilation Successful for main.cpp!

--- LINKING OBJECT FILES ---
✅ Linking Successful! Executable: solution
Detected 'cin' in main.cpp.
Please provide *all* required inputs for the C++ program below, one value per line.
To signal that you have provided all inputs, enter an empty line (just press Enter) after a prompt.
Alternatively, you can press Ctrl+D for EOF (End Of File) to finish input.
C++ Input (type empty line to finish)> 67
C++ Input (type empty line to finish)> 

--- RUNNING PROGRAM ---
Enter your age: Your age is: 67


--- CLEANING UP ---
Removed executable: solution
Removed source file: main.cpp
Removed object file: main.o


---
## **5. Operators & Problem Solving**

### **Problem 1: Sum of Two Numbers**

In [5]:

%%cpp
#include <iostream>
using namespace std;

int main() {
    int a, b;
    cout << "Enter first number: ";
    cin >> a;
    cout << "Enter second number: ";
    cin >> b;

    int sum = a + b;
    cout << "Sum is: " << sum << endl;
    return 0;
}


--- COMPILING C++ FILES ---
✅ Compilation Successful for main.cpp!

--- LINKING OBJECT FILES ---
✅ Linking Successful! Executable: solution
Detected 'cin' in main.cpp.
Please provide *all* required inputs for the C++ program below, one value per line.
To signal that you have provided all inputs, enter an empty line (just press Enter) after a prompt.
Alternatively, you can press Ctrl+D for EOF (End Of File) to finish input.
C++ Input (type empty line to finish)> 34
C++ Input (type empty line to finish)> 987
C++ Input (type empty line to finish)> 

--- RUNNING PROGRAM ---
Enter first number: Enter second number: Sum is: 1021


--- CLEANING UP ---
Removed executable: solution
Removed source file: main.cpp
Removed object file: main.o


### **Problem 2: Unary Operator Logic (Pre vs Post Increment)**
* `a++`: Use then increment.
* `++a`: Increment then use.

In [ ]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    int a = 10;
    int b = a++; // Post-increment: b gets 10, THEN a becomes 11

    cout << "After b = a++ :" << endl;
    cout << "b = " << b << " (Old value)" << endl;
    cout << "a = " << a << " (Updated value)" << endl;

    int x = 10;
    int y = ++x; // Pre-increment: x becomes 11 FIRST, then y gets 11

    cout << "After y = ++x :" << endl;
    cout << "y = " << y << " (Updated value)" << endl;
    cout << "x = " << x << " (Updated value)" << endl;

    return 0;
}

In [ ]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    int a, b;
    cout << "Enter first number: ";
    cin >> a;
    cout << "Enter second number: ";
    cin >> b;

    int sum = a + b;
    cout << "Sum is: " << sum << endl;
    return 0;
}
---INPUT---
5
10

---
## **Homework: Calculator**
Performs +, -, *, / on two numbers.

In [ ]:
%%cpp
#include<iostream>
using namespace std;

int main () {

    int a;
    cout << "Enter the value of a: "; // Prompt for input
    cin >> a; // Read into variable a

    float b; // Correct declaration
    cout << "Enter the value of b: "; // Prompt for input
    cin >> b; // Read into variable b

    int c; // Correct declaration
    cout << "Enter the value of c: "; // Prompt for input
    cin >> c; // Read into variable c

    cout << "Values: " << a << ", " << b << ", " << c << endl;
    cout << "Sum: " << a + b + c << endl;
    cout << "Difference: " << a - b - c << endl;
    cout << "Multiplication: " << a * b * c << endl;
    // Ensure b is not zero to prevent division by zero
    if (b != 0) {
        cout << "Division (a/b): " << a / b << endl; // Assuming a/b as example
    } else {
        cout << "Cannot divide by zero (b is 0)." << endl;
    }

    return 0;
}

#Lecture no 3 Conditional Statements and Loops

###**else if ladder syntax**


---

if (condition1) {
  
}

else if (condition2) {

}

else if (condition3) {

}

else {

}

In [ ]:
if (condition1) {
}
else if (condition2) {
}
else if (condition3) {
}
else {
}

###**Nested if–else**
An if–else inside another if–else.


In [ ]:
if (condition1) {
    if (condition2) {
    }
    else {
    }
}
else {
}


###**Logical OR (||) Conditions**

In [ ]:
if (condition1 || condition2)

###**Logical NOT (!) Conditions**

if (!condition)

{

  //statement

}

###**Using if–else to control loops**


In [ ]:
for (int i = 1; i <= 10; i++) {
    if (i == 6) {
        break;
    }
    cout << i << " ";
}


#### **Example: Positive or Negative Number**

The following code checks if a number is positive or negative. A number is considered positive if it is greater than or equal to zero [[02:44](http://www.youtube.com/watch?v=qR9U6bKxJ7g&t=164)].

In [ ]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    int n = 45;

    if (n >= 0) {
        cout << "n is positive" << endl;
    } else {
        cout << "n is negative" << endl;
    }

    return 0;
}


In [12]:
%%cpp
#include <iostream>
using namespace std;
int main(){
    int a, b, c; // Corrected: removed extra comma
    cout <<"Enter first number: "; // Added prompt for clarity
    cin >>a;
    cout <<"Enter second number: "; // Added prompt for clarity
    cin >>b;
    cout <<"Enter third number: "; // Added prompt for clarity
    cin >>c;

    if (a % 2 == 0 && b % 2 == 0 && c % 2 == 0) { // Corrected: if syntax, modulo operator, and added curly braces
        cout <<"All numbers are even" << endl; // Added endl for formatting, changed message
    } else {
        cout <<"At least one number is odd" << endl; // Added endl for formatting, changed message
    }
    return 0; // Added return 0 for proper main function termination
}

--- COMPILING C++ FILES ---
✅ Compilation Successful for main.cpp!

--- LINKING OBJECT FILES ---
✅ Linking Successful! Executable: solution
Detected 'cin' in main.cpp.
Please provide *all* required inputs for the C++ program below, one value per line.
To signal that you have provided all inputs, enter an empty line (just press Enter) after a prompt.
Alternatively, you can press Ctrl+D for EOF (End Of File) to finish input.
C++ Input (type empty line to finish)> 4567
C++ Input (type empty line to finish)> 87654
C++ Input (type empty line to finish)> 34567890
C++ Input (type empty line to finish)> 

--- RUNNING PROGRAM ---
Enter first number: Enter second number: Enter third number: At least one number is odd


--- CLEANING UP ---
Removed executable: solution
Removed source file: main.cpp
Removed object file: main.o


#### **Example: Voting System**

A simple program to determine if a user is eligible to vote based on their age (threshold is 18) [[08:12](http://www.youtube.com/watch?v=qR9U6bKxJ7g&t=492)].

In [ ]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    int age;
    cout << "Enter age: ";
    cin >> age;

    if (age >= 18) {
        cout << "You can vote" << endl;
    } else {
        cout << "You can't vote" << endl;
    }

    return 0;
}


#### **Example: Odd or Even Number**

Using the modulo operator (`%`), we check if a number is divisible by 2 to determine if it is even or odd [[10:35](http://www.youtube.com/watch?v=qR9U6bKxJ7g&t=635)].

In [ ]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    int n;
    cout << "Enter number: ";
    cin >> n;

    if (n % 2 == 0) {
        cout << "Even" << endl;
    } else {
        cout << "Odd" << endl;
    }

    return 0;
}


---

### **2. Else-If and Grading System**

When there are more than two possibilities, `else if` is used to check multiple conditions sequentially [[12:54](http://www.youtube.com/watch?v=qR9U6bKxJ7g&t=774)].

**Grading Logic:**

* Marks  90: Grade A
* Marks 80 to 89: Grade B
* Marks < 80: Grade C

In [ ]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    int marks;
    cout << "Enter marks: ";
    cin >> marks;

    if (marks >= 90) {
        cout << "A" << endl;
    } else if (marks >= 80 && marks < 90) {
        cout << "B" << endl;
    } else {
        cout << "C" << endl;
    }

    return 0;
}


---

### **3. Character Case Check**

This program identifies if a character is lowercase or uppercase using ASCII ranges or direct character comparison [[20:05](http://www.youtube.com/watch?v=qR9U6bKxJ7g&t=1205)].

In [ ]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    char ch;
    cout << "Enter character: ";
    cin >> ch;

    if (ch >= 'a' && ch <= 'z') {
        cout << "Lower case" << endl;
    } else if (ch >= 'A' && ch <= 'Z') {
        cout << "Upper case" << endl;
    }

    return 0;
}


---

### **4. Ternary Operator**

The ternary operator is a shorthand for simple `if-else` statements, written as: `condition ? expression1 : expression2;` [[26:01](http://www.youtube.com/watch?v=qR9U6bKxJ7g&t=1561)].

In [16]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    int n ;
    cout <<"enter the number:- ";
    cin >> n ;
    cout << (n >= 0 ? "positive" : "negative") << endl;
    return 0;
}


--- COMPILING C++ FILES ---
✅ Compilation Successful for main.cpp!

--- LINKING OBJECT FILES ---
✅ Linking Successful! Executable: solution
Detected 'cin' in main.cpp.
Please provide *all* required inputs for the C++ program below, one value per line.
To signal that you have provided all inputs, enter an empty line (just press Enter) after a prompt.
Alternatively, you can press Ctrl+D for EOF (End Of File) to finish input.
C++ Input (type empty line to finish)> 45
C++ Input (type empty line to finish)> 

--- RUNNING PROGRAM ---
enter the number:- positive


--- CLEANING UP ---
Removed executable: solution
Removed source file: main.cpp
Removed object file: main.o


---

### **5. Loops in C++**

Loops are used to repeat a block of code multiple times. The video covers `while`, `for`, and `do-while` loops [[28:47](http://www.youtube.com/watch?v=qR9U6bKxJ7g&t=1727)].

## 1. What a loop really is (fact)

A loop is a **control structure that repeats a block of code** as long as a condition remains `true`.

A loop has **three mandatory parts** (explicit or implicit):

1. **Initialization** – where repetition starts
2. **Condition** – decides whether to continue
3. **Update** – changes state to eventually stop the loop

If any of these is wrong → **infinite loop or logic bug**.

---

## 2. `for` loop

### Syntax

```cpp
for (initialization; condition; update) {
    // repeated code
}
```

### Example

```cpp
for (int i = 1; i <= 5; i++) {
    cout << i << " ";
}
```

### Execution order

1. Initialization → once
2. Condition → before every iteration
3. Body executes
4. Update executes
5. Repeat

---

### When to use `for`

✔ Known number of iterations
✔ Counter-based loops
✔ Clean and readable logic

---

## 3. `while` loop

### Syntax

```cpp
//initilisation
while (condition) {
    // repeated code
    //update
}
```

### Example

```cpp
int i = 1;

while (i <= 5) {
    cout << i << " ";
    i++;
}
```

### Key fact

* Condition is checked **before** execution
* Loop may run **zero times**

---

### When to use `while`

✔ Unknown number of iterations
✔ Input-driven loops
✔ Condition-controlled repetition

---

## 4. `do–while` loop

### Syntax

```cpp
do {
    // repeated code
} while (condition);
```

### Example

```cpp
int i = 1;

do {
    cout << i << " ";
    i++;
} while (i <= 5);
```

### Key fact

* Runs **at least once**
* Condition checked **after** execution

---

### When to use `do–while`

✔ Menu-driven programs
✔ Guaranteed one execution

---

## 5. Nested loops (loop inside loop)

### Syntax

```cpp
for (int i = 1; i <= 3; i++) {
    for (int j = 1; j <= 3; j++) {
        cout << i << j << " ";
    }
}
```

### Output

```
11 12 13 21 22 23 31 32 33
```

### Rule

* Inner loop completes **fully** for each outer iteration

---

## 6. Infinite loops (intentional & accidental)

### Intentional

```cpp
while (true) {
    // runs forever until break
}
```

### Accidental (BUG)

```cpp
int i = 1;
while (i <= 5) {
    cout << i;
    // missing i++
}
```

---

### 7. Loop control statements

### `break` — exits loop completely

```cpp
for (int i = 1; i <= 10; i++) {
    if (i == 5)
        break;
    cout << i << " ";
}
```

Output:

```
1 2 3 4
```

---

### `continue` — skips current iteration

```cpp
for (int i = 1; i <= 5; i++) {
    if (i == 3)
        continue;
    cout << i << " ";
}
```

Output:

```
1 2 4 5
```

---

### 8. Loop with `if–else` (very common)

```cpp
for (int i = 1; i <= 5; i++) {
    if (i % 2 == 0)
        cout << i << " Even\n";
    else
        cout << i << " Odd\n";
}
```

---

### 10. Comparison of all loops

| Loop | Condition check | Minimum runs | Best use |
| --- | --- | --- | --- |
| `for` | Before | 0 | Known iterations |
| `while` | Before | 0 | Unknown iterations |
| `do–while` | After | 1 | Menus |

---

## 11. Real-world loop examples

### Sum of numbers

```cpp
int sum = 0;
for (int i = 1; i <= 10; i++) {
    sum += i;
}
```

---

### Factorial

```cpp
int n = 5, fact = 1;
for (int i = 1; i <= n; i++) {
    fact *= i;
}
```

---

### Reverse a number

```cpp
int n = 123, rev = 0;

while (n != 0) {
    rev = rev * 10 + n % 10;
    n /= 10;
}
```

#### **While Loop: Print Numbers 1 to N**

[[37:06](http://www.youtube.com/watch?v=qR9U6bKxJ7g&t=2226)]

In [ ]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    int n = 10;
    int count = 1;
    while (count <= n) {
        cout << count << " ";
        count++;
    }
    cout << endl;
    return 0;
}


#### **For Loop: Calculate Sum from 1 to N**

The `for` loop combines initialization, condition, and updation in one line [[43:18](http://www.youtube.com/watch?v=qR9U6bKxJ7g&t=2598)].

In [ ]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    int n = 5;
    int sum = 0;
    for (int i = 1; i <= n; i++) {
        sum += i;
    }
    cout << "Sum = " << sum << endl;
    return 0;
}


#### **Break Statement**

The `break` keyword immediately terminates the loop when a certain condition is met [[54:29](http://www.youtube.com/watch?v=qR9U6bKxJ7g&t=3269)].

---

### **6. Practical Logic: Check Prime Number**

A number is prime if it is only divisible by 1 and itself. An optimized approach involves checking divisibility up to  [[01:06:40](http://www.youtube.com/watch?v=qR9U6bKxJ7g&t=4000)].

In [ ]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    int n = 7;
    bool isPrime = true;

    for (int i = 2; i * i <= n; i++) {
        if (n % i == 0) {
            isPrime = false;
            break;
        }
    }

    if (isPrime) {
        cout << "Prime number" << endl;
    } else {
        cout << "Non-prime number" << endl;
    }

    return 0;
}


---

### **7. Nested Loops Introduction**

Nested loops (a loop inside another loop) are introduced for pattern printing [[01:22:20](http://www.youtube.com/watch?v=qR9U6bKxJ7g&t=4940)].

In [ ]:
%%cpp
#include <iostream>
using namespace std;

int main() {
    int n = 5; // Number of lines
    int m = 5; // Stars per line

    for (int i = 1; i <= n; i++) {
        for (int j = 1; j <= m; j++) {
            cout << "*";
        }
        cout << endl;
    }

    return 0;
}







#  Lecture no 4 .Square Pattern


#### **Variation A: Numbers (1 2 3 4)**

In [ ]:
%%writefile square_pattern_numbers.cpp
#include <iostream>
using namespace std;

int main() {
    int n = 4;
    for (int i = 1; i <= n; i++) { // outer loop
        for (int j = 1; j <= n; j++) { // inner loop
            cout << j << " ";
        }
        cout << endl;
    }
    return 0;
}

!g++ square_pattern_numbers.cpp -o square_pattern_numbers
!./square_pattern_numbers

#### **Variation B: Stars (* * * *)**

In [ ]:
%%writefile square_pattern_stars.cpp
#include <iostream>
using namespace std;

int main() {
    int n = 4;
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            cout << "*" << " ";
        }
        cout << endl;
    }
    return 0;
}

!g++ square_pattern_stars.cpp -o square_pattern_stars
!./square_pattern_stars

#### **Variation C: Characters (A B C D)**

Characters are stored internally as ASCII values. Adding 1 to 'A' results in 'B' after type conversion [[15:02](http://www.youtube.com/watch?v=rga_q2N7vU8&t=902)].

In [ ]:
%%writefile square_pattern_chars.cpp
#include <iostream>
using namespace std;

int main() {
    int n = 4;
    for (int i = 0; i < n; i++) {
        char ch = 'A';
        for (int j = 0; j < n; j++) {
            cout << ch << " ";
            ch = ch + 1; // updates character
        }
        cout << endl;
    }
    return 0;
}

!g++ square_pattern_chars.cpp -o square_pattern_chars
!./square_pattern_chars

---

### **2. Continuous Number Square Pattern**

Numbers do not reset at the start of a new line. They continue from the previous value [[21:45](http://www.youtube.com/watch?v=rga_q2N7vU8&t=1305)].

In [ ]:
%%writefile continuous_num_square.cpp
#include <iostream>
using namespace std;

int main() {
    int n = 3;
    int num = 1;
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            cout << num << " ";
            num++;
        }
        cout << endl;
    }
    return 0;
}

!g++ continuous_num_square.cpp -o continuous_num_square
!./continuous_num_square

---

### **3. Triangle Patterns**

In triangular patterns, the number of columns printed in a row depends on the row index () [[35:04](http://www.youtube.com/watch?v=rga_q2N7vU8&t=2104)].

#### **Variation A: Star Triangle**

In [ ]:
%%writefile star_triangle.cpp
#include <iostream>
using namespace std;

int main() {
    int n = 4;
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < i + 1; j++) {
            cout << "* ";
        }
        cout << endl;
    }
    return 0;
}

!g++ star_triangle.cpp -o star_triangle
!./star_triangle

#### **Variation B: Same-Number Triangle**

In [ ]:
%%writefile same_number_triangle.cpp
#include <iostream>
using namespace std;

int main() {
    int n = 4;
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < i + 1; j++) {
            cout << (i + 1) << " ";
        }
        cout << endl;
    }
    return 0;
}

!g++ same_number_triangle.cpp -o same_number_triangle
!./same_number_triangle

#### **Variation C: Sequential Number Triangle**

In [ ]:
%%writefile sequential_num_triangle.cpp
#include <iostream>
using namespace std;

int main() {
    int n = 4;
    for (int i = 0; i < n; i++) {
        for (int j = 1; j <= i + 1; j++) {
            cout << j << " ";
        }
        cout << endl;
    }
    return 0;
}

---

### **4. Reverse Triangle Pattern**

Printing numbers in decreasing order within each row [[49:55](http://www.youtube.com/watch?v=rga_q2N7vU8&t=2995)].

In [ ]:
%%writefile reverse_triangle.cpp
#include <iostream>
using namespace std;

int main() {
    int n = 4;
    for (int i = 0; i < n; i++) {
        for (int j = i + 1; j > 0; j--) {
            cout << j << " ";
        }
        cout << endl;
    }
    return 0;
}

!g++ reverse_triangle.cpp -o reverse_triangle
!./reverse_triangle

---

### **5. Floyd’s Triangle Pattern**

A triangular pattern where numbers are continuous and do not reset [[53:21](http://www.youtube.com/watch?v=rga_q2N7vU8&t=3201)].

In [ ]:
%%writefile floyds_triangle.cpp
#include <iostream>
using namespace std;

int main() {
    int n = 4;
    int num = 1;
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < i + 1; j++) {
            cout << num << " ";
            num++;
        }
        cout << endl;
    }
    return 0;
}

!g++ floyds_triangle.cpp -o floyds_triangle
!./floyds_triangle

---

### **6. Inverted Triangle Pattern**

This pattern involves a combination of leading spaces and numbers [[56:23](http://www.youtube.com/watch?v=rga_q2N7vU8&t=3383)].

*   **Spaces:**  spaces per row.
*   **Numbers:**  numbers per row.

In [ ]:
%%writefile inverted_triangle.cpp
#include <iostream>
using namespace std;

int main() {
    int n = 4;
    for (int i = 0; i < n; i++) {
        // Spaces
        for (int j = 0; j < i; j++) {
            cout << " ";
        }
        // Numbers
        for (int j = 0; j < n - i; j++) {
            cout << (i + 1);
        }
        cout << endl;
    }
    return 0;
}

!g++ inverted_triangle.cpp -o inverted_triangle
!./inverted_triangle

---

### **7. Pyramid Pattern**

A complex pattern broken into three parts per row: leading spaces, increasing numbers, and decreasing numbers [[01:03:25](http://www.youtube.com/watch?v=rga_q2N7vU8&t=3805)].

1.  **Spaces:**
2.  **Increasing Numbers:** 1 to
3.  **Decreasing Numbers:**  to 1

In [ ]:
%%writefile pyramid_pattern.cpp
#include <iostream>
using namespace std;

int main() {
    int n = 4;
    for (int i = 0; i < n; i++) {
        // Spaces
        for (int j = 0; j < n - i - 1; j++) {
            cout << " ";
        }
        // Increasing Numbers
        for (int j = 1; j <= i + 1; j++) {
            cout << j;
        }
        // Decreasing Numbers
        for (int j = i; j >= 1; j--) {
            cout << j;
        }
        cout << endl;
    }
    return 0;
}

!g++ pyramid_pattern.cpp -o pyramid_pattern
!./pyramid_pattern

---

### **8. Hollow Diamond Pattern**

The pattern is divided into a **Top Part** (Hollow Pyramid) and a **Bottom Part** (Inverted Hollow Pyramid) [[01:13:28](http://www.youtube.com/watch?v=rga_q2N7vU8&t=4408)].

In [ ]:
%%writefile hollow_diamond.cpp
#include <iostream>
using namespace std;

int main() {
    int n = 4;

    // Top Part
    for (int i = 0; i < n; i++) {
        // Spaces
        for (int j = 0; j < n - i - 1; j++) {
            cout << " ";
        }
        cout << "*";
        if (i != 0) {
            // Inner Spaces
            for (int j = 0; j < 2 * i - 1; j++) {
                cout << " ";
            }
            cout << "*";
        }
        cout << endl;
    }

    // Bottom Part
    for (int i = 0; i < n - 1; i++) {
        // Spaces
        for (int j = 0; j < i + 1; j++) {
            cout << " ";
        }
        cout << "*";
        if (i != n - 2) {
            // Inner Spaces
            for (int j = 0; j < 2 * (n - i - 2) - 1; j++) {
                cout << " ";
            }
            cout << "*";
        }
        cout << endl;
    }

    return 0;
}

!g++ hollow_diamond.cpp -o hollow_diamond
!./hollow_diamond



# Lecture no 5 Functions

A function that only prints something and does not return a value uses the `void` return type.

In [ ]:
%%cpp
#include <iostream>
using namespace std;

// Function Definition
void printHello() {
    cout << "Hello\n";
}

int main() {
    // Function Call / Invocation
    printHello();
    printHello();
    return 0;
}

---

### **3. Sum of Two Numbers (Parameters and Return Type)**

Parameters are the variables used in the function definition. Arguments are the actual values passed during the call.

In [ ]:
%%cpp
#include <iostream>
using namespace std;

// Function to calculate sum
int sum(int a, int b) {
    int s = a + b;
    return s;
}

int main() {
    cout << "Sum is: " << sum(10, 5) << endl; // 10 and 5 are arguments
    return 0;
}

---

### **4. Minimum of Two Numbers**

In [ ]:
%%cpp
#include <iostream>
using namespace std;

int minOfTwo(int a, int b) { // Parameters
    if (a < b) {
        return a;
    } else {
        return b;
    }
}

int main() {
    cout << "Min = " << minOfTwo(5, 3) << endl; // Arguments
    return 0;
}

---

### **5. Calculating Sum from 1 to N**

In [ ]:
%%cpp
#include <iostream>
using namespace std;

int sumN(int n) {
    int sum = 0;
    for(int i=1; i<=n; i++) {
        sum += i;
    }
    return sum;
}

---

### **6. Calculating N Factorial**

Factorial of  () is the product of all numbers from 1 to .

In [ ]:
%%cpp
#include <iostream>
using namespace std;

int factorial(int n) {
    int fact = 1;
    for(int i=1; i<=n; i++) {
        fact *= i;
    }
    return fact;
}

int main() {
    cout << "Factorial of 4: " << factorial(4) << endl;
    cout << "Factorial of 5: " << factorial(5) << endl;
    return 0;
}

---

### **7. Memory Management: Call Stack**

When a function is called, a **Stack Frame** is created in the **Stack Memory**.

*   The `main` function is the starting point and enters the stack first.
*   New functions are added to the top of the stack.
*   Once a function finishes (reaches `return`), its frame is destroyed, and control returns to the previous function.

---

### **8. Pass by Value**

In C++, parameters are passed by value. This means a **copy** of the argument is created in the function's memory. Changes made inside the function do not affect the original variable in `main`.

In [ ]:
%%cpp
#include <iostream>
using namespace std;

void changeX(int x) {
    x = 2 * x;
    cout << "x in function: " << x << endl;
}

int main() {
    int x = 5;
    changeX(x);
    cout << "x in main: " << x << endl; // Remains 5
    return 0;
}

---

### **9. Sum of Digits of a Number**

**Approach:** Use `% 10` to extract the last digit and `/ 10` to remove it until the number is 0.

In [ ]:
%%cpp
#include <iostream>
using namespace std;

int sumOfDigits(int num) {
    int digSum = 0;
    while(num > 0) {
        int lastDig = num % 10;
        num /= 10;
        digSum += lastDig;
    }
    return digSum;
}

int main() {
    cout << "Sum of digits of 2356: " << sumOfDigits(2356) << endl;
    return 0;
}

---

### **10. Binomial Coefficient (nCr)**

Formula: . This demonstrates a function calling another function (`factorial`).

In [ ]:
%%cpp
#include <iostream>
using namespace std;

int factorial(int n) {
    int fact = 1;
    for(int i=1; i<=n; i++) {
        fact *= i;
    }
    return fact;
}

int nCr(int n, int r) {
    int fact_n = factorial(n);
    int fact_r = factorial(r);
    int fact_nmr = factorial(n - r);

    return fact_n / (fact_r * fact_nmr);
}

int main() {
    int n = 8, r = 2;
    cout << "8C2 = " << nCr(n, r) << endl; // Output: 28
    return 0;
}

---

### **11. Homework Problems**

1.  **Prime Check:** Write a function to check if a number is prime.
2.  **Print Primes:** Write a function to print all prime numbers from 2 to .
3.  **Fibonacci:** Write a function to print the -th Fibonacci term.
4.  **Switch Statement:** Explore the `switch` statement for conditional cases.

**Summary of Concepts:**

*   Functions provide modularity and code reusability.
*   Understand **Pass by Value** vs. **Call Stack**.
*   **Litterals:** Fixed values (e.g., `15`, `'A'`) that don't change.

---

*Video Title: Lecture 5: Functions | DSA Series by Shradha Khapra Ma'am | C++*
*Source: Apna College YouTube*